In [2]:
import os
from dotenv import load_dotenv
from llama_index.llms.openrouter import OpenRouter

load_dotenv()

OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables. Please check your .env file.")

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
     print("Warning: GOOGLE_API_KEY not found. Google Embedding might fail if not using ADC.") # Or raise error



/home/mutaician/AIEngineerLearning/HuggingFaceAgentsCourse/llamaindex/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
# llm initialise

llm = OpenRouter(
    api_key=OPENROUTER_API_KEY,
    max_tokens=256,
    model="openrouter/optimus-alpha"
)

In [4]:
response = llm.complete("Hello, how are you?")
print(response)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


Components  - Helps to prepare, find and use relevant information to help complete tasks
can be used as RAG

QueryEngine - key component for building agentic RAG workflows to create a RAG pipeline

RAQ pipeline using componets 
1. Loading - getting data from where it lives
2. Indexing - creating data structure for querying the data (vector embeddings)
3. storing - store index and metadata
4. Querying - using the data indexes by llms 
5. evaluation - checking effectiveness


In [5]:
# load data
from datasets import load_dataset
from pathlib import Path


dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

In [6]:
# use SmpleDirectoryReader to load person descriptions from the data directory

from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

In [7]:
# Use ingestionpipeline to create nodes from the documents and prepare them for QueryEngine 
# split the documents into smaller chunks and embed them

from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create pipeline
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        GoogleGenAIEmbedding(model_name="text-embedding-004", api_key=GOOGLE_API_KEY)
    ]
)

# run pipeline
nodes = await pipeline.arun(documents=documents[:10])
nodes

[TextNode(id_='b33c483d-819a-4e2c-aafa-ff48c57e9972', embedding=[0.017838376, 0.004960109, -0.015247883, 0.015447473, 0.021288533, 0.0017558996, -0.0054151663, 0.03317173, 0.017536657, 0.020825867, 0.014706515, 0.03208674, 0.050930902, 0.008202867, 0.021288782, -0.020926679, 0.022979619, 0.076361835, -0.070216924, 0.0058998256, 0.02268513, -0.0263135, 0.047885425, -0.074613646, -0.019523839, -0.011186321, 0.025170486, -0.063492894, 0.045871608, -0.022110183, 0.07984705, 0.057684436, 0.033961747, -0.044686094, -0.04401283, 0.047121283, -0.0039002812, -0.03258355, 0.037435986, -0.08935929, -0.04329128, 0.019677594, 0.0048755854, 0.06721369, -0.024956977, -0.02988329, 0.006202784, 0.031517055, -0.033431668, 0.04473984, 0.05191161, 0.023901125, -0.0647585, 0.020354854, -0.0170539, -0.014000553, -0.01754647, -0.016486645, 0.038728155, -0.002907086, 0.03708068, 0.011219051, -0.02459299, -0.059771, -0.015954997, -0.035527017, -0.0005118873, 0.009595176, -0.06237783, 0.045915794, -0.018928748,

In [8]:
# storing and indexing documents

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        GoogleGenAIEmbedding(model_name="text-embedding-004", api_key=GOOGLE_API_KEY)
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

10

In [9]:
from llama_index.core import VectorStoreIndex

embed_model = GoogleGenAIEmbedding(model_name="text-embedding-004", api_key=GOOGLE_API_KEY)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

In [10]:
# Querying the index
import nest_asyncio

nest_asyncio.apply()  # This is needed to run the query engine
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query(
    "Respond using a persona that describes computer experiences?"
)
response

Response(response="As someone who's really focused on web development, I spend a lot of time working with HTML and CSS to build and style websites. Most of my computer experiences revolve around coding in text editors, previewing my work in browsers, and troubleshooting layout issues. I enjoy experimenting with different design ideas and learning new techniques to make my websites look better and function smoothly. Every day, I’m getting more comfortable navigating development tools and understanding how the pieces of a website fit together.", source_nodes=[NodeWithScore(node=TextNode(id_='954035b2-b080-4bd1-a868-167096258b77', embedding=None, metadata={'file_path': '/home/mutaician/AIEngineerLearning/HuggingFaceAgentsCourse/llamaindex/data/persona_1001.txt', 'file_name': 'persona_1001.txt', 'file_type': 'text/plain', 'file_size': 157, 'creation_date': '2025-04-12', 'last_modified_date': '2025-04-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date'

In [11]:
# evaluation and observability

from llama_index.core.evaluation import FaithfulnessEvaluator

# query index
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

True

In [ ]:

import llama_index

PHOENIX_API_KEY = os.environ.get("PHOENIX_API_KEY")
if not PHOENIX_API_KEY:
    raise ValueError("PHOENIX_API_KEY not found in environment variables. Please check your .env file.")

os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)

Attempting to instrument while already instrumented


In [ ]:
response = query_engine.query(
    "Respond like a person who is interested in space   "
)
response

Response(response="I'm absolutely fascinated by the mysteries of the universe! The idea of exploring distant planets, understanding black holes, and searching for signs of life beyond Earth really excites me. I love keeping up with the latest space missions and discoveries—there's always something new and awe-inspiring happening out there. The vastness of space makes me curious about our place in the cosmos and what the future of space exploration might hold.", source_nodes=[NodeWithScore(node=TextNode(id_='d3375d1b-cc33-4a73-b31a-562ea5b43b70', embedding=None, metadata={'file_path': '/home/mutaician/AIEngineerLearning/HuggingFaceAgentsCourse/llamaindex/data/persona_1004.txt', 'file_name': 'persona_1004.txt', 'file_type': 'text/plain', 'file_size': 160, 'creation_date': '2025-04-12', 'last_modified_date': '2025-04-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_nam

In [17]:
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

False